# 单Agent

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pocketflow import Node,Flow

load_dotenv()

True

In [4]:
import yaml

In [3]:
def call_llm(prompt):
    client = OpenAI(
        base_url= os.getenv("BASE_URL"),
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    res = client.chat.completions.create(
        model = os.getenv("MODEL_NAME"),
        messages = [{
            "role": "user",
            "content": prompt
        }]
    )
    return res.choices[0].message.content

In [5]:
# 行动决定节点
class DecideAction(Node):
    def prep(self,shared):
        context = shared.get("context","没有之前的搜索")
        travel_query = shared['travel_query']
        return travel_query,context
    
    def exec(self,inputs):
        travel_query,context = inputs
        prompt = f"""
        ### 上下文  
        你是一个旅行规划助手，可以搜索网络获取信息。  
        旅行查询: {travel_query}  
        已有信息: {context}  
        
        ### 可用行动  
        [1] search  
        描述: 在网络上查找更多信息  
        参数:  
            - query (str): 要搜索的内容  
        
        [2] answer  
        描述: 用当前知识回答问题  
        参数:  
            - answer (str): 给出的旅行建议  
        
        ## 下一步行动  
        根据上下文和可用行动决定下一步。
        按以下格式返回:  
        
        ```yaml  
        thinking: |  
            <你的逐步思考过程>  
        action: search 或 answer  
        reason: <为什么选择这个行动>  
        answer: <如果行动是answer>
        search_query: <如果行动是search，具体的搜索查询>
        ```"""

        res = call_llm(prompt)
        yaml_str = res.split("```yaml")[1].split("```")[0].strip()
        return yaml.safe_load(yaml_str)
    
    def post(self,shared,prep_str,exec_str):
        if exec_str['action'] == "search":
            shared['search_query'] = exec_str['search_query']
        else:
            shared['context'] = exec_str['answer']
        
        return exec_str["action"]

In [ ]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key="tavily api key")

def search_web(query):
    res = tavily_client.search(query)
    results_str = "\n\n".join([
        f"标题: {r['title']}\n网址: {r['url']}\n摘要: {r['content']}"
        for r in res['results']
    ])
    return results_str

In [7]:
# 网络搜索节点
class SearchWeb(Node):
    def prep(self,shared):
        return shared['search_query']
    def exec(self,search_query):
        print(f"🌐 正在搜索: {search_query}")  
        res = search_web(search_query)
        return res
    def post(self,shared,prep_str,exec_str):
        previous = shared.get("context","")
        shared['context'] =  previous + "\n\n搜索: " + shared["search_query"] + "\n结果: " + exec_str 
        print(f"📚 找到信息，正在分析结果...")  
        return "decide"

In [8]:
class AnswerQuestion(Node):
    def prep(self,shared):
        return shared['travel_query'],shared.get("context","")
    
    def exec(self,inputs):
        travel_query,context = inputs
        print(f"✍️ 正在制定旅行建议...")  
        prompt = f"""  
        ### 上下文  
        基于以下信息，回答旅行查询。  
        旅行查询: {travel_query}  
        搜集的信息: {context}  
        
        ## 你的建议:  
        提供一个全面的旅行建议，包括:  
        1. 目的地概览  
        2. 推荐的景点  
        3. 最佳旅行时间  
        4. 交通和住宿建议  
        5. 预算考虑  
        """     
        res = call_llm(prompt)  
        return res
    def post(self,shared,prep_str,exec_str):
        shared['answer'] = exec_str
        print(f"✅ 旅行建议生成成功")  
        return "done"

In [ ]:
def create_travel_agent_flow():
    decide_node = DecideAction()
    search_node = SearchWeb()
    answer_node = AnswerQuestion()

    # 表示当 decide_node 节点的决策结果为 "search" 时，工作流将跳转到 search_node 节点继续执行
    decide_node  - "search" >> search_node  

    # 表示当 decide_node 节点的决策结果为 "answer" 时，工作流将跳转到 answer_node 节点继续执行
    decide_node  - "answer" >> answer_node
    
    # 表示当 search_node 节点执行完毕后，返回 "decide" 信号，工作流将回到 decide_node 节点重新进行决策
    search_node  - "decide" >> decide_node

    return Flow(start=decide_node)

In [ ]:

def main():
    shared = {
        "travel_query": "我想去日本旅行，帮我规划一下"
    }
    travel_agent_flow = create_travel_agent_flow()
    travel_agent_flow.run(shared)
    print("\n🎯 旅行建议:")  
    print(shared.get("answer", "没有找到建议"))

if __name__ == '__main__':
    main()